In [42]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
import pickle

In [3]:
df = pd.read_csv('spam.csv',encoding="latin-1")

In [4]:
df.shape
df = df.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"] , axis=1)
df = df.rename(columns={"v1":"label","v2":"text"})
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
df['length'] = df['text'].apply(len)
df.head()

,label,text,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [6]:
df.loc[:,'label']=df.label.map({'ham':0,'spam':1})
df.head()

,label,text,length
0,0,"Go until jurong point, crazy.. Available only ...",111
1,0,Ok lar... Joking wif u oni...,29
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,0,U dun say so early hor... U c already then say...,49
4,0,"Nah I don't think he goes to usf, he lives aro...",61


In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [46]:
count = CountVectorizer()
text = count.fit_transform(df['text'])
input = ["Congratulations! You have won a cash prize of $1,000,000. To claim your prize, reply to this email with your full name, address, and bank account details. Act quickly to secure your winnings."]
input_transformed = count.transform(input)

In [66]:
# Split the data into features and labels
X = df['text']
y = df['label']

# Create CountVectorizer
count_vectorizer = CountVectorizer()

# Initialize variables for best F1 score and confusion matrix
best_f1_score = 0
best_confusion_matrix = None
best_classification_report = None

# Initialize lists for storing metrics
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

# Perform k-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in kfold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Transform text data into numerical features
    X_train_transformed = count_vectorizer.fit_transform(X_train)
    X_test_transformed = count_vectorizer.transform(X_test)

    # Train the classifier
    mnb = MultinomialNB()
    mnb.fit(X_train_transformed, y_train)

    # Make predictions on the test set
    y_pred = mnb.predict(X_test_transformed)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Append metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

    # Calculate F1 score
    f1 = f1_score(y_test, y_pred)

    # Calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Generate classification report
    report = classification_report(y_test, y_pred)

    # Check if current fold has a better F1 score
    if f1 > best_f1_score:
        best_f1_score = f1
        best_confusion_matrix = cm
        best_classification_report = report

print("KFold for MNB")

# Print the best F1 score, confusion matrix, and classification report
print("Best F1 Score:", best_f1_score)
print("Best Confusion Matrix:")
print(best_confusion_matrix)
print("Best Classification Report:")
print(best_classification_report)

# Calculate mean metrics
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)

# Create a table for the results
results_table = pd.DataFrame({
    'Accuracy': accuracy_scores,
    'Precision': precision_scores,
    'Recall': recall_scores,
    'F1 Score': f1_scores
})

print("\nResults Table:")
print(results_table)

# Calculate mean metrics
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)

# Print mean metrics
print("\nMean Accuracy:", mean_accuracy)
print("Mean Precision:", mean_precision)
print("Mean Recall:", mean_recall)

input_text = ["Congratulations! You have won a cash prize of $1,000,000. To claim your prize, reply to this email with your full name, address, and bank account details. Act quickly to secure your winnings."]

input_transformed = count_vectorizer.transform(input_text)
predictions = mnb.predict(input_transformed)
print("Predictions:", predictions)

pickle.dump(mnb,open('./kfold/mnb_kfold.pkl','wb'))

KFold for MNB
Best F1 Score: 0.9634146341463415
Best Confusion Matrix:
[[472   1]
 [  5  79]]
Best Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       473
           1       0.99      0.94      0.96        84

    accuracy                           0.99       557
   macro avg       0.99      0.97      0.98       557
weighted avg       0.99      0.99      0.99       557


Results Table:
   Accuracy  Precision    Recall  F1 Score
0  0.989247   0.969231  0.940299  0.954545
1  0.978495   0.986301  0.867470  0.923077
2  0.985637   0.984375  0.900000  0.940299
3  0.985637   0.949367  0.949367  0.949367
4  0.989228   0.948718  0.973684  0.961039
5  0.983842   0.984375  0.887324  0.933333
6  0.989228   1.000000  0.921053  0.958904
7  0.989228   0.969697  0.941176  0.955224
8  0.989228   0.987500  0.940476  0.963415
9  0.980251   0.942857  0.904110  0.923077

Mean Accuracy: 0.9860022007297156
Mean Precision: 0.972242128

In [67]:
# Split the data into features and labels
X = df['text']
y = df['label']

# Create CountVectorizer
count_vectorizer = CountVectorizer()

# Initialize variables for best F1 score and confusion matrix
best_f1_score = 0
best_confusion_matrix = None
best_classification_report = None

# Initialize lists for storing metrics
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

# Perform k-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in kfold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Transform text data into numerical features
    X_train_transformed = count_vectorizer.fit_transform(X_train)
    X_test_transformed = count_vectorizer.transform(X_test)

    # Train the classifier
    svc = SVC(kernel='sigmoid', gamma=1.0)
    svc.fit(X_train_transformed, y_train)

    # Make predictions on the test set
    y_pred = svc.predict(X_test_transformed)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Append metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

    # Calculate F1 score
    f1 = f1_score(y_test, y_pred)

    # Calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Generate classification report
    report = classification_report(y_test, y_pred)

    # Check if current fold has a better F1 score
    if f1 > best_f1_score:
        best_f1_score = f1
        best_confusion_matrix = cm
        best_classification_report = report

print("KFold for SVC")

# Print the best F1 score, confusion matrix, and classification report
print("Best F1 Score:", best_f1_score)
print("Best Confusion Matrix:")
print(best_confusion_matrix)
print("Best Classification Report:")
print(best_classification_report)

# Calculate mean metrics
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)

# Create a table for the results
results_table = pd.DataFrame({
    'Accuracy': accuracy_scores,
    'Precision': precision_scores,
    'Recall': recall_scores,
    'F1 Score': f1_scores
})

print("\nResults Table:")
print(results_table)

# Calculate mean metrics
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)

# Print mean metrics
print("\nMean Accuracy:", mean_accuracy)
print("Mean Precision:", mean_precision)
print("Mean Recall:", mean_recall)

input_text = ["Congratulations! You have won a cash prize of $1,000,000. To claim your prize, reply to this email with your full name, address, and bank account details. Act quickly to secure your winnings."]

input_transformed = count_vectorizer.transform(input_text)
predictions = svc.predict(input_transformed)
print("Predictions:", predictions)
pickle.dump(svc,open('./kfold/svc_kfold.pkl','wb'))

KFold for SVC
Best F1 Score: 0.48366013071895425
Best Confusion Matrix:
[[441  45]
 [ 34  37]]
Best Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.91      0.92       486
           1       0.45      0.52      0.48        71

    accuracy                           0.86       557
   macro avg       0.69      0.71      0.70       557
weighted avg       0.87      0.86      0.86       557


Results Table:
   Accuracy  Precision    Recall  F1 Score
0  0.853047   0.402597  0.462687  0.430556
1  0.831541   0.400000  0.265060  0.318841
2  0.829443   0.349398  0.414286  0.379085
3  0.843806   0.447368  0.430380  0.438710
4  0.831239   0.381579  0.381579  0.381579
5  0.858169   0.451220  0.521127  0.483660
6  0.818671   0.313433  0.276316  0.293706
7  0.859964   0.419355  0.382353  0.400000
8  0.834829   0.448718  0.416667  0.432099
9  0.820467   0.324675  0.342466  0.333333

Mean Accuracy: 0.8381176682560826
Mean Precision: 0.39383428

In [68]:
# Split the data into features and labels
X = df['text']
y = df['label']

# Create CountVectorizer
count_vectorizer = CountVectorizer()

# Initialize variables for best F1 score and confusion matrix
best_f1_score = 0
best_confusion_matrix = None
best_classification_report = None

# Initialize lists for storing metrics
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

# Perform k-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in kfold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Transform text data into numerical features
    X_train_transformed = count_vectorizer.fit_transform(X_train)
    X_test_transformed = count_vectorizer.transform(X_test)

    # Train the classifier
    knc = KNeighborsClassifier()

    knc.fit(X_train_transformed, y_train)

    # Make predictions on the test set
    y_pred = knc.predict(X_test_transformed)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Append metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

    # Calculate F1 score
    f1 = f1_score(y_test, y_pred)

    # Calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Generate classification report
    report = classification_report(y_test, y_pred)

    # Check if current fold has a better F1 score
    if f1 > best_f1_score:
        best_f1_score = f1
        best_confusion_matrix = cm
        best_classification_report = report

print("KFold for KNC")

# Print the best F1 score, confusion matrix, and classification report
print("Best F1 Score:", best_f1_score)
print("Best Confusion Matrix:")
print(best_confusion_matrix)
print("Best Classification Report:")
print(best_classification_report)

# Calculate mean metrics
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)

# Create a table for the results
results_table = pd.DataFrame({
    'Accuracy': accuracy_scores,
    'Precision': precision_scores,
    'Recall': recall_scores,
    'F1 Score': f1_scores
})

print("\nResults Table:")
print(results_table)

# Calculate mean metrics
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)

# Print mean metrics
print("\nMean Accuracy:", mean_accuracy)
print("Mean Precision:", mean_precision)
print("Mean Recall:", mean_recall)
input_text = ["Congratulations! You have won a cash prize of $1,000,000. To claim your prize, reply to this email with your full name, address, and bank account details. Act quickly to secure your winnings."]

input_transformed = count_vectorizer.transform(input_text)
predictions = knc.predict(input_transformed)
print("Predictions:", predictions)

pickle.dump(knc,open('./kfold/knc_kfold.pkl','wb'))


KFold for KNC
Best F1 Score: 0.6851851851851851
Best Confusion Matrix:
[[486   0]
 [ 34  37]]
Best Classification Report:
              precision    recall  f1-score   support

           0       0.93      1.00      0.97       486
           1       1.00      0.52      0.69        71

    accuracy                           0.94       557
   macro avg       0.97      0.76      0.83       557
weighted avg       0.94      0.94      0.93       557


Results Table:
   Accuracy  Precision    Recall  F1 Score
0  0.939068        1.0  0.492537  0.660000
1  0.899642        1.0  0.325301  0.490909
2  0.931777        1.0  0.457143  0.627451
3  0.929982        1.0  0.506329  0.672269
4  0.919210        1.0  0.407895  0.579439
5  0.938959        1.0  0.521127  0.685185
6  0.921005        1.0  0.421053  0.592593
7  0.924596        1.0  0.382353  0.553191
8  0.897666        1.0  0.321429  0.486486
9  0.913824        1.0  0.342466  0.510204

Mean Accuracy: 0.9215729426072856
Mean Precision: 1.0
Mean Re

In [69]:
# Split the data into features and labels
X = df['text']
y = df['label']

# Create CountVectorizer
count_vectorizer = CountVectorizer()

# Initialize variables for best F1 score and confusion matrix
best_f1_score = 0
best_confusion_matrix = None
best_classification_report = None

# Initialize lists for storing metrics
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

# Perform k-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in kfold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Transform text data into numerical features
    X_train_transformed = count_vectorizer.fit_transform(X_train)
    X_test_transformed = count_vectorizer.transform(X_test)

    # Train the classifier
    dtc = DecisionTreeClassifier(max_depth=5)

    dtc.fit(X_train_transformed, y_train)

    # Make predictions on the test set
    y_pred = dtc.predict(X_test_transformed)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Append metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

    # Calculate F1 score
    f1 = f1_score(y_test, y_pred)

    # Calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Generate classification report
    report = classification_report(y_test, y_pred)

    # Check if current fold has a better F1 score
    if f1 > best_f1_score:
        best_f1_score = f1
        best_confusion_matrix = cm
        best_classification_report = report

print("KFold for DTC")

# Print the best F1 score, confusion matrix, and classification report
print("Best F1 Score:", best_f1_score)
print("Best Confusion Matrix:")
print(best_confusion_matrix)
print("Best Classification Report:")
print(best_classification_report)

# Calculate mean metrics
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)

# Create a table for the results
results_table = pd.DataFrame({
    'Accuracy': accuracy_scores,
    'Precision': precision_scores,
    'Recall': recall_scores,
    'F1 Score': f1_scores
})

print("\nResults Table:")
print(results_table)

# Calculate mean metrics
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)

# Print mean metrics
print("\nMean Accuracy:", mean_accuracy)
print("Mean Precision:", mean_precision)
print("Mean Recall:", mean_recall)
input_text = ["Congratulations! You have won a cash prize of $1,000,000. To claim your prize, reply to this email with your full name, address, and bank account details. Act quickly to secure your winnings."]

input_transformed = count_vectorizer.transform(input_text)
predictions = dtc.predict(input_transformed)
print("Predictions:", predictions)

pickle.dump(dtc,open('./kfold/dtc_kfold.pkl','wb'))


KFold for DTC
Best F1 Score: 0.8374999999999999
Best Confusion Matrix:
[[464  14]
 [ 12  67]]
Best Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       478
           1       0.83      0.85      0.84        79

    accuracy                           0.95       557
   macro avg       0.90      0.91      0.91       557
weighted avg       0.95      0.95      0.95       557


Results Table:
   Accuracy  Precision    Recall  F1 Score
0  0.962366   0.910714  0.761194  0.829268
1  0.944444   0.919355  0.686747  0.786207
2  0.940754   0.803279  0.700000  0.748092
3  0.953321   0.827160  0.848101  0.837500
4  0.942549   0.854839  0.697368  0.768116
5  0.944345   0.822581  0.718310  0.766917
6  0.942549   0.940000  0.618421  0.746032
7  0.947935   0.842105  0.705882  0.768000
8  0.942549   0.882353  0.714286  0.789474
9  0.942549   0.859649  0.671233  0.753846

Mean Accuracy: 0.9463362998140319
Mean Precision: 0.866203498

In [70]:
# Split the data into features and labels
X = df['text']
y = df['label']

# Create CountVectorizer
count_vectorizer = CountVectorizer()

# input = ["Congratulations! You have won a cash prize of $1,000,000. To claim your prize, reply to this email with your full name, address, and bank account details. Act quickly to secure your winnings."]
# input_transformed = count_vectorizer.transform(input)

# Initialize variables for best F1 score and confusion matrix
best_f1_score = 0
best_confusion_matrix = None
best_classification_report = None

# Initialize lists for storing metrics
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

# Perform k-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in kfold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Transform text data into numerical features
    X_train_transformed = count_vectorizer.fit_transform(X_train)
    X_test_transformed = count_vectorizer.transform(X_test)

    # Train the classifier
    lrc = LogisticRegression(solver='liblinear', penalty='l1')


    lrc.fit(X_train_transformed, y_train)

    # Make predictions on the test set
    y_pred = lrc.predict(X_test_transformed)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Append metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

    # Calculate F1 score
    f1 = f1_score(y_test, y_pred)

    # Calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Generate classification report
    report = classification_report(y_test, y_pred)

    # Check if current fold has a better F1 score
    if f1 > best_f1_score:
        best_f1_score = f1
        best_confusion_matrix = cm
        best_classification_report = report

print("KFold for LRC")

# Print the best F1 score, confusion matrix, and classification report
print("Best F1 Score:", best_f1_score)
print("Best Confusion Matrix:")
print(best_confusion_matrix)
print("Best Classification Report:")
print(best_classification_report)

# Calculate mean metrics
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)

# Create a table for the results
results_table = pd.DataFrame({
    'Accuracy': accuracy_scores,
    'Precision': precision_scores,
    'Recall': recall_scores,
    'F1 Score': f1_scores
})

print("\nResults Table:")
print(results_table)

# Calculate mean metrics
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)

# Print mean metrics
print("\nMean Accuracy:", mean_accuracy)
print("Mean Precision:", mean_precision)
print("Mean Recall:", mean_recall)

# lrc.predict(input_transformed)

pickle.dump(lrc,open('lrc_KFold.pkl','wb'))

input_text = ["you win 5000$ in luck draw"]

input_transformed = count_vectorizer.transform(input_text)
predictions = lrc.predict(input_transformed)
print("Predictions:", predictions)
pickle.dump(count_vectorizer,open('vectorizer.pkl','wb'))
pickle.dump(lrc,open('./kfold/lrc_kfold.pkl','wb'))


KFold for LRC
Best F1 Score: 0.9710144927536231
Best Confusion Matrix:
[[486   0]
 [  4  67]]
Best Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       486
           1       1.00      0.94      0.97        71

    accuracy                           0.99       557
   macro avg       1.00      0.97      0.98       557
weighted avg       0.99      0.99      0.99       557


Results Table:
   Accuracy  Precision    Recall  F1 Score
0  0.983871   1.000000  0.865672  0.928000
1  0.967742   0.957746  0.819277  0.883117
2  0.982047   0.983871  0.871429  0.924242
3  0.983842   0.926829  0.962025  0.944099
4  0.985637   0.972222  0.921053  0.945946
5  0.992819   1.000000  0.943662  0.971014
6  0.980251   0.971014  0.881579  0.924138
7  0.982047   0.983333  0.867647  0.921875
8  0.976661   0.938272  0.904762  0.921212
9  0.973070   0.953125  0.835616  0.890511

Mean Accuracy: 0.9807986332310186
Mean Precision: 0.968641336

In [71]:

# Split the data into features and labels
X = df['text']
y = df['label']

# Create CountVectorizer
count_vectorizer = CountVectorizer()

# Initialize variables for best F1 score and confusion matrix
best_f1_score = 0
best_confusion_matrix = None
best_classification_report = None

# Initialize lists for storing metrics
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

# Perform k-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in kfold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Transform text data into numerical features
    X_train_transformed = count_vectorizer.fit_transform(X_train)
    X_test_transformed = count_vectorizer.transform(X_test)

    # Train the classifier
    rfc = RandomForestClassifier(n_estimators=50, random_state=2)


    rfc.fit(X_train_transformed, y_train)

    # Make predictions on the test set
    y_pred = rfc.predict(X_test_transformed)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Append metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

    # Calculate F1 score
    f1 = f1_score(y_test, y_pred)

    # Calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Generate classification report
    report = classification_report(y_test, y_pred)

    # Check if current fold has a better F1 score
    if f1 > best_f1_score:
        best_f1_score = f1
        best_confusion_matrix = cm
        best_classification_report = report

print("KFold for RFC")

# Print the best F1 score, confusion matrix, and classification report
print("Best F1 Score:", best_f1_score)
print("Best Confusion Matrix:")
print(best_confusion_matrix)
print("Best Classification Report:")
print(best_classification_report)

# Calculate mean metrics
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)

# Create a table for the results
results_table = pd.DataFrame({
    'Accuracy': accuracy_scores,
    'Precision': precision_scores,
    'Recall': recall_scores,
    'F1 Score': f1_scores
})

print("\nResults Table:")
print(results_table)

# Calculate mean metrics
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)

# Print mean metrics
print("\nMean Accuracy:", mean_accuracy)
print("Mean Precision:", mean_precision)
print("Mean Recall:", mean_recall)
input_text = ["Congratulations! You have won a cash prize of $1,000,000. To claim your prize, reply to this email with your full name, address, and bank account details. Act quickly to secure your winnings."]

input_transformed = count_vectorizer.transform(input_text)
predictions = rfc.predict(input_transformed)
print("Predictions:", predictions)

pickle.dump(rfc,open('./kfold/rfc_kfold.pkl','wb'))


KFold for RFC
Best F1 Score: 0.9395973154362416
Best Confusion Matrix:
[[478   0]
 [  9  70]]
Best Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       478
           1       1.00      0.89      0.94        79

    accuracy                           0.98       557
   macro avg       0.99      0.94      0.97       557
weighted avg       0.98      0.98      0.98       557


Results Table:
   Accuracy  Precision    Recall  F1 Score
0  0.982079        1.0  0.850746  0.919355
1  0.967742        1.0  0.783133  0.878378
2  0.980251        1.0  0.842857  0.914729
3  0.983842        1.0  0.886076  0.939597
4  0.978456        1.0  0.842105  0.914286
5  0.978456        1.0  0.830986  0.907692
6  0.974865        1.0  0.815789  0.898551
7  0.978456        1.0  0.823529  0.903226
8  0.978456        1.0  0.857143  0.923077
9  0.976661        1.0  0.821918  0.902256

Mean Accuracy: 0.9779264235568169
Mean Precision: 1.0
Mean Re

In [72]:

# Split the data into features and labels
X = df['text']
y = df['label']

# Create CountVectorizer
count_vectorizer = CountVectorizer()

# Initialize variables for best F1 score and confusion matrix
best_f1_score = 0
best_confusion_matrix = None
best_classification_report = None

# Initialize lists for storing metrics
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

# Perform k-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in kfold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Transform text data into numerical features
    X_train_transformed = count_vectorizer.fit_transform(X_train)
    X_test_transformed = count_vectorizer.transform(X_test)

    # Train the classifier
    abc = AdaBoostClassifier(n_estimators=50, random_state=2)



    abc.fit(X_train_transformed, y_train)

    # Make predictions on the test set
    y_pred = abc.predict(X_test_transformed)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Append metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

    # Calculate F1 score
    f1 = f1_score(y_test, y_pred)

    # Calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Generate classification report
    report = classification_report(y_test, y_pred)

    # Check if current fold has a better F1 score
    if f1 > best_f1_score:
        best_f1_score = f1
        best_confusion_matrix = cm
        best_classification_report = report

print("KFold for ABC")

# Print the best F1 score, confusion matrix, and classification report
print("Best F1 Score:", best_f1_score)
print("Best Confusion Matrix:")
print(best_confusion_matrix)
print("Best Classification Report:")
print(best_classification_report)

# Calculate mean metrics
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)

# Create a table for the results
results_table = pd.DataFrame({
    'Accuracy': accuracy_scores,
    'Precision': precision_scores,
    'Recall': recall_scores,
    'F1 Score': f1_scores
})

print("\nResults Table:")
print(results_table)

# Calculate mean metrics
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)

# Print mean metrics
print("\nMean Accuracy:", mean_accuracy)
print("Mean Precision:", mean_precision)
print("Mean Recall:", mean_recall)
input_text = ["Congratulations! You have won a cash prize of $1,000,000. To claim your prize, reply to this email with your full name, address, and bank account details. Act quickly to secure your winnings."]

input_transformed = count_vectorizer.transform(input_text)
predictions = abc.predict(input_transformed)
print("Predictions:", predictions)
pickle.dump(abc,open('./kfold/abc_kfold.pkl','wb'))



KFold for ABC
Best F1 Score: 0.9565217391304348
Best Confusion Matrix:
[[485   1]
 [  5  66]]
Best Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       486
           1       0.99      0.93      0.96        71

    accuracy                           0.99       557
   macro avg       0.99      0.96      0.98       557
weighted avg       0.99      0.99      0.99       557


Results Table:
   Accuracy  Precision    Recall  F1 Score
0  0.974910   0.981818  0.805970  0.885246
1  0.967742   0.933333  0.843373  0.886076
2  0.964093   0.903226  0.800000  0.848485
3  0.976661   0.934211  0.898734  0.916129
4  0.978456   0.944444  0.894737  0.918919
5  0.989228   0.985075  0.929577  0.956522
6  0.969479   0.983607  0.789474  0.875912
7  0.974865   0.921875  0.867647  0.893939
8  0.969479   0.913580  0.880952  0.896970
9  0.965889   0.950000  0.780822  0.857143

Mean Accuracy: 0.9730803137648565
Mean Precision: 0.945116872

In [73]:

# Split the data into features and labels
X = df['text']
y = df['label']

# Create CountVectorizer
count_vectorizer = CountVectorizer()

# Initialize variables for best F1 score and confusion matrix
best_f1_score = 0
best_confusion_matrix = None
best_classification_report = None

# Initialize lists for storing metrics
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

# Perform k-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in kfold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Transform text data into numerical features
    X_train_transformed = count_vectorizer.fit_transform(X_train)
    X_test_transformed = count_vectorizer.transform(X_test)

    # Train the classifier
    bc = BaggingClassifier(n_estimators=50, random_state=2)




    bc.fit(X_train_transformed, y_train)

    # Make predictions on the test set
    y_pred = bc.predict(X_test_transformed)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Append metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

    # Calculate F1 score
    f1 = f1_score(y_test, y_pred)

    # Calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Generate classification report
    report = classification_report(y_test, y_pred)

    # Check if current fold has a better F1 score
    if f1 > best_f1_score:
        best_f1_score = f1
        best_confusion_matrix = cm
        best_classification_report = report

print("KFold for BC")

# Print the best F1 score, confusion matrix, and classification report
print("Best F1 Score:", best_f1_score)
print("Best Confusion Matrix:")
print(best_confusion_matrix)
print("Best Classification Report:")
print(best_classification_report)

# Calculate mean metrics
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)

# Create a table for the results
results_table = pd.DataFrame({
    'Accuracy': accuracy_scores,
    'Precision': precision_scores,
    'Recall': recall_scores,
    'F1 Score': f1_scores
})

print("\nResults Table:")
print(results_table)

# Calculate mean metrics
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)

# Print mean metrics
print("\nMean Accuracy:", mean_accuracy)
print("Mean Precision:", mean_precision)
print("Mean Recall:", mean_recall)

input_text = ["Congratulations! You have won a cash prize of $1,000,000. To claim your prize, reply to this email with your full name, address, and bank account details. Act quickly to secure your winnings."]

input_transformed = count_vectorizer.transform(input_text)
predictions = bc.predict(input_transformed)
print("Predictions:", predictions)
pickle.dump(bc,open('./kfold/bc_kfold.pkl','wb'))



KFold for BC
Best F1 Score: 0.9393939393939394
Best Confusion Matrix:
[[488   3]
 [  5  62]]
Best Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       491
           1       0.95      0.93      0.94        67

    accuracy                           0.99       558
   macro avg       0.97      0.96      0.97       558
weighted avg       0.99      0.99      0.99       558


Results Table:
   Accuracy  Precision    Recall  F1 Score
0  0.985663   0.953846  0.925373  0.939394
1  0.962366   0.907895  0.831325  0.867925
2  0.962298   0.901639  0.785714  0.839695
3  0.980251   0.935897  0.924051  0.929936
4  0.976661   0.909091  0.921053  0.915033
5  0.982047   0.942029  0.915493  0.928571
6  0.973070   0.969231  0.828947  0.893617
7  0.973070   0.934426  0.838235  0.883721
8  0.974865   0.926829  0.904762  0.915663
9  0.973070   0.983333  0.808219  0.887218

Mean Accuracy: 0.9743360810280368
Mean Precision: 0.9364217165

In [74]:




# Split the data into features and labels
X = df['text']
y = df['label']

# Create CountVectorizer
count_vectorizer = CountVectorizer()

# Initialize variables for best F1 score and confusion matrix
best_f1_score = 0
best_confusion_matrix = None
best_classification_report = None

# Initialize lists for storing metrics
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

# Perform k-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in kfold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Transform text data into numerical features
    X_train_transformed = count_vectorizer.fit_transform(X_train)
    X_test_transformed = count_vectorizer.transform(X_test)

    # Train the classifier
    etc = ExtraTreesClassifier(n_estimators=50, random_state=2)




    etc.fit(X_train_transformed, y_train)

    # Make predictions on the test set
    y_pred = etc.predict(X_test_transformed)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Append metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

    # Calculate F1 score
    f1 = f1_score(y_test, y_pred)

    # Calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Generate classification report
    report = classification_report(y_test, y_pred)

    # Check if current fold has a better F1 score
    if f1 > best_f1_score:
        best_f1_score = f1
        best_confusion_matrix = cm
        best_classification_report = report

print("KFold for ETC")

# Print the best F1 score, confusion matrix, and classification report
print("Best F1 Score:", best_f1_score)
print("Best Confusion Matrix:")
print(best_confusion_matrix)
print("Best Classification Report:")
print(best_classification_report)

# Calculate mean metrics
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)

# Create a table for the results
results_table = pd.DataFrame({
    'Accuracy': accuracy_scores,
    'Precision': precision_scores,
    'Recall': recall_scores,
    'F1 Score': f1_scores
})

print("\nResults Table:")
print(results_table)

# Calculate mean metrics
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)

# Print mean metrics
print("\nMean Accuracy:", mean_accuracy)
print("Mean Precision:", mean_precision)
print("Mean Recall:", mean_recall)

input_text = ["Congratulations! You have won a cash prize of $1,000,000. To claim your prize, reply to this email with your full name, address, and bank account details. Act quickly to secure your winnings."]

input_transformed = count_vectorizer.transform(input_text)
predictions = etc.predict(input_transformed)
print("Predictions:", predictions)

pickle.dump(etc,open('./kfold/etc_kfold.pkl','wb'))



KFold for ETC
Best F1 Score: 0.9664429530201343
Best Confusion Matrix:
[[480   1]
 [  4  72]]
Best Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       481
           1       0.99      0.95      0.97        76

    accuracy                           0.99       557
   macro avg       0.99      0.97      0.98       557
weighted avg       0.99      0.99      0.99       557


Results Table:
   Accuracy  Precision    Recall  F1 Score
0  0.989247   1.000000  0.910448  0.953125
1  0.971326   0.985507  0.819277  0.894737
2  0.983842   1.000000  0.871429  0.931298
3  0.983842   1.000000  0.886076  0.939597
4  0.991023   0.986301  0.947368  0.966443
5  0.983842   1.000000  0.873239  0.932331
6  0.978456   1.000000  0.842105  0.914286
7  0.987433   1.000000  0.897059  0.945736
8  0.976661   1.000000  0.845238  0.916129
9  0.973070   1.000000  0.794521  0.885496

Mean Accuracy: 0.9818742237923335
Mean Precision: 0.997180861

In [75]:





# Split the data into features and labels
X = df['text']
y = df['label']

# Create CountVectorizer
count_vectorizer = CountVectorizer()

# Initialize variables for best F1 score and confusion matrix
best_f1_score = 0
best_confusion_matrix = None
best_classification_report = None

# Initialize lists for storing metrics
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

# Perform k-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in kfold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Transform text data into numerical features
    X_train_transformed = count_vectorizer.fit_transform(X_train)
    X_test_transformed = count_vectorizer.transform(X_test)

    # Train the classifier
    gbdt = GradientBoostingClassifier(n_estimators=50,random_state=2)



    gbdt.fit(X_train_transformed, y_train)

    # Make predictions on the test set
    y_pred = gbdt.predict(X_test_transformed)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Append metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

    # Calculate F1 score
    f1 = f1_score(y_test, y_pred)

    # Calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Generate classification report
    report = classification_report(y_test, y_pred)

    # Check if current fold has a better F1 score
    if f1 > best_f1_score:
        best_f1_score = f1
        best_confusion_matrix = cm
        best_classification_report = report

print("KFold for GBDT")

# Print the best F1 score, confusion matrix, and classification report
print("Best F1 Score:", best_f1_score)
print("Best Confusion Matrix:")
print(best_confusion_matrix)
print("Best Classification Report:")
print(best_classification_report)

# Calculate mean metrics
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)

# Create a table for the results
results_table = pd.DataFrame({
    'Accuracy': accuracy_scores,
    'Precision': precision_scores,
    'Recall': recall_scores,
    'F1 Score': f1_scores
})

print("\nResults Table:")
print(results_table)

# Calculate mean metrics
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)

# Print mean metrics
print("\nMean Accuracy:", mean_accuracy)
print("Mean Precision:", mean_precision)
print("Mean Recall:", mean_recall)
input_text = ["Congratulations! You have won a cash prize of $1,000,000. To claim your prize, reply to this email with your full name, address, and bank account details. Act quickly to secure your winnings."]

input_transformed = count_vectorizer.transform(input_text)
predictions = gbdt.predict(input_transformed)
print("Predictions:", predictions)

pickle.dump(gbdt,open('./kfold/gbdt_kfold.pkl','wb'))




KFold for GBDT
Best F1 Score: 0.8750000000000001
Best Confusion Matrix:
[[476   2]
 [ 16  63]]
Best Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       478
           1       0.97      0.80      0.88        79

    accuracy                           0.97       557
   macro avg       0.97      0.90      0.93       557
weighted avg       0.97      0.97      0.97       557


Results Table:
   Accuracy  Precision    Recall  F1 Score
0  0.967742   0.980392  0.746269  0.847458
1  0.949821   0.966102  0.686747  0.802817
2  0.958707   0.912281  0.742857  0.818898
3  0.967684   0.969231  0.797468  0.875000
4  0.965889   0.901408  0.842105  0.870748
5  0.965889   0.919355  0.802817  0.857143
6  0.960503   0.965517  0.736842  0.835821
7  0.969479   0.981132  0.764706  0.859504
8  0.965889   1.000000  0.773810  0.872483
9  0.953321   0.943396  0.684932  0.793651

Mean Accuracy: 0.9624923585773761
Mean Precision: 0.95388141

In [76]:
# Split the data into features and labels
X = df['text']
y = df['label']

# Create CountVectorizer
count_vectorizer = CountVectorizer()

# Transform text data into numerical features
X_transformed = count_vectorizer.fit_transform(X)

# Initialize variables for best F1 score and confusion matrix
best_f1_score = 0
best_confusion_matrix = None
best_classification_report = None

# Initialize lists for storing metrics
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

# Perform k-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in kfold.split(X_transformed):
    X_train, X_test = X_transformed[train_index], X_transformed[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the classifier
    mnb = MultinomialNB()
    mnb.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = mnb.predict(X_test)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Append metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

    # Calculate F1 score
    f1 = f1_score(y_test, y_pred)

    # Calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Generate classification report
    report = classification_report(y_test, y_pred)

    # Check if current fold has a better F1 score
    if f1 > best_f1_score:
        best_f1_score = f1
        best_confusion_matrix = cm
        best_classification_report = report

print("KFold for MNB")

# Print the best F1 score, confusion matrix, and classification report
print("Best F1 Score:", best_f1_score)
print("Best Confusion Matrix:")
print(best_confusion_matrix)
print("Best Classification Report:")
print(best_classification_report)

# Calculate mean metrics
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)

# Create a table for the results
results_table = pd.DataFrame({
    'Accuracy': accuracy_scores,
    'Precision': precision_scores,
    'Recall': recall_scores,
    'F1 Score': f1_scores
})

print("\nResults Table:")
print(results_table)

# Calculate mean metrics
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)

# Print mean metrics
print("\nMean Accuracy:", mean_accuracy)
print("Mean Precision:", mean_precision)
print("Mean Recall:", mean_recall)

# Make predictions on new input
input_text = ["Congratulations! You have won a cash prize of $1,000,000. To claim your prize, reply to this email with your full name, address, and bank account details. Act quickly to secure your winnings."]

input_transformed = count_vectorizer.transform(input_text)
predictions = mnb.predict(input_transformed)
print("Predictions:", predictions)

pickle.dump(mnb,open('./kfold/mnb_kfold.pkl','wb'))



KFold for MNB
Best F1 Score: 0.9605263157894737
Best Confusion Matrix:
[[478   3]
 [  3  73]]
Best Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       481
           1       0.96      0.96      0.96        76

    accuracy                           0.99       557
   macro avg       0.98      0.98      0.98       557
weighted avg       0.99      0.99      0.99       557


Results Table:
   Accuracy  Precision    Recall  F1 Score
0  0.980287   0.878378  0.970149  0.921986
1  0.978495   0.961039  0.891566  0.925000
2  0.978456   0.902778  0.928571  0.915493
3  0.985637   0.938272  0.962025  0.950000
4  0.987433   0.936709  0.973684  0.954839
5  0.983842   0.930556  0.943662  0.937063
6  0.989228   0.960526  0.960526  0.960526
7  0.983842   0.904110  0.970588  0.936170
8  0.982047   0.940476  0.940476  0.940476
9  0.962298   0.817073  0.917808  0.864516

Mean Accuracy: 0.9811564126818659
Mean Precision: 0.916991640